# Lancia Tuning Basic Calculators

In [1]:
using Unitful
using UnitfulUS
using Statistics

In [2]:
# Register jupyter notebook as a source of units 
# for new units we define
Unitful.register(Main);
@unit HP "HP" Horsepower 0.7456999*u"kW" false

HP

In [3]:
lancia_ccs = 1756u"cm^3";
lancia_cylinders = 4;
lancia_injector_maxf = 158u"cm^3/minute";

In [4]:
uconvert(u"gal_us/hr", lancia_injector_maxf)

395000000//157725491 galᵘˢ hr^-1

In [21]:
stoic_afr_93 = 14.68;
injector_pres_nom = 3u"bar";
gasoline_mass = 6.259u"lb/gal_us"
uconvert(u"psi", injector_pres_nom)

43.51132131906277 psi

In [54]:
gas_lbs_cc(lbs) = u"cm^3/minute"(lbs / gasoline_mass);
gas_cc_lbs(cc) = u"gal_us/hr"(cc) * gasoline_mass;

# Injector Calculator

BSFC = brake specific fuel consumption - effeciency measure per horsepower of engine

## Min Injector size @Stock pressure

In [6]:
function fuel_needs(power::Unitful.Power, turbo=false)
    power_hp = uconvert(u"HP", power);
    if(turbo)
        BSFC_turbo = mean([0.55u"lb/(HP*hr)", 0.7u"lb/(HP*hr)"]);
        BSFC_turbo * power_hp;
    else
        BSFC_na = mean([0.3u"lb/(HP*hr)", 0.5u"lb/(HP*hr)"]);
        BSFC_na * power_hp;
    end
end

fuel_needs (generic function with 2 methods)

In [7]:
function injector_size(power::Unitful.Power, cylinders, turbo=false, margin=0.2) 
    return fuel_needs(power, turbo) * (1+margin) / cylinders
end

injector_size (generic function with 3 methods)

In [17]:
inj_size_min = injector_size(150u"HP", lancia_cylinders, true)

28.125 lb hr^-1

In [62]:
min_inj_size(power::Unitful.Power) = injector_size(power, 4, true, 0.2) |> gas_lbs_cc

min_inj_size (generic function with 1 method)

In [70]:
min_inj_size(160u"HP")

302.39749033391917 cm^3 minute^-1

## Fuel Pressure Change Calculation

In [27]:
final_pressure = 4u"bar"

4 bar

In [29]:
flow_rate(pressure::Unitful.Pressure, nom_flow) = nom_flow * sqrt(pressure / injector_pres_nom)

flow_rate (generic function with 1 method)

In [61]:
flow_rate(4u"bar", 158u"cm^3/minute" |> gas_cc_lbs) |> gas_lbs_cc

182.44268506392174 cm^3 minute^-1